In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
!pip install psycopg2

In [ ]:
#store crime json data into DataFram

crime_df = pd.read_json("data/cpd-incidents.json")
#crime_df.head()
crime_df.head()

In [ ]:


crime_df = crime_df[["id","year","crimeday", "crime_type", "lat", "lon"]]
crime_df.head()

In [ ]:
#drop na values 
crime_df = crime_df.dropna()
#crime_df.info()
#convert year from float to int.
crime_df['year'] = crime_df['year'].apply(np.int64)
crime_df.info()
crime_df.head()

In [ ]:
#create database connection
username = 'postgres'
password = '.'
#removed password for privacy purposes
host = '127.0.0.1'
port = '5432'
database = 'crime_incidents'

db_url = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, database)

engine = create_engine(db_url)
conn = engine.connect()

engine.table_names()

#use pandas to load CSV converted DataFrame into database
crime_df.to_sql(name='crimes', con=engine, if_exists='append', index=False)

#read in streetlights json data.  Get rid of unwanted rows.  Convert lat_lon json object to columns.

In [ ]:
lights_df = pd.read_json("data/streetlights.json")
#lights_df.head()
lights_df = lights_df[["facilityid", "geo_point_2d", "address", "areaname"]]

lights_df = lights_df.rename(columns = {"geo_point_2d" : "lat_lon"})
lights_df.head()


# Unpack lat_lon cloumn.  ie convert dict tp series, then concatenat to df.  dropping oringinal lat_lon column.
lights_df = pd.concat([lights_df.drop(['lat_lon'], axis=1), lights_df['lat_lon'].apply(pd.Series)], axis=1)
lights_df.head()

In [ ]:
lights_df.columns

In [ ]:
lights_df.head()

In [ ]:
#create database connection
username = 'postgres'
password = 'AlbionBio1'
#removed password for privacy purposes
host = '127.0.0.1'
port = '5432'
database = 'crime_incidents'

db_url = 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, database)

engine = create_engine(db_url)
conn = engine.connect()

engine.table_names()

#use pandas to load CSV converted DataFrame into database
lights_df.to_sql(name='streetlights', con=engine, if_exists='append', index=False)

#read in streetlights json data.  Get rid of unwanted rows.  Convert lat_lon json object to columns.

In [6]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

engine = create_engine('sqlite:///crimes.sqlite', echo=True)
sqlite_connection = engine.connect()